### Validation Protocols for Hay Model

* Validation Protocols have been added in stimvalidation.py and multimodalfitting/ecode/config/protocols.json contains these along with optimization protocols of Hay Model.
* The validation protocols added and there parameters names are mentioned below

1. **sAHP**: "long_amplitude", "amplitude" and "holding_amplitude" 
2. **HyperDepol**: "hyperpol_amplitude", "depol_amplitude" and "holding_amplitude" 
3. **PosCheops**: "ramp1_amp" "ramp2_amp", "ramp3_amp" and "holding_amplitude"
4. **NoiseOU3**: "filename". Currently, it reads "multimodalfitting/ecode/NoiseOU3txt.txt" which has (time, current) data and injects this stimulus in the soma 

All the validation protocols are injected in soma.
If 'NoiseOU3' protocol is added in protocols.json, it takes ~ 1 hr to simulate. Removing it will produce the desired figures for the other protocols in ~ 1 min.


In [1]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time

import sys

from pathlib import Path

%matplotlib notebook

In [2]:
!rm -r x86_64
!nrnivmodl mechanisms

which: no xcrun in (/gpfs/bbp.cscs.ch/home/mandge/lfpenv/bin:/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/externals/2021-01-06/linux-rhel7-x86_64/gcc-9.3.0/hpe-mpi-2.22.hmpt-r52ypu/bin:/gpfs/bbp.cscs.ch/ssd/apps/hpc/jenkins/deploy/externals/2021-01-06/linux-rhel7-x86_64/gcc-9.3.0/python-3.8.3-suxrst/bin:/gpfs/bbp.cscs.ch/apps/rhscl/rh-nodejs10/root/usr/bin:/gpfs/bbp.cscs.ch/ssd/apps/tools/jupyter/jupyterlab/bin:/opt/clmgr/sbin:/opt/clmgr/bin:/opt/sgi/sbin:/opt/sgi/bin:/usr/lib64/qt-3.3/bin:/gpfs/bbp.cscs.ch/home/mandge//.nix-profile/bin:/nix/var/nix/profiles/default/bin:/usr/lib64/ccache:/usr/local/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/gpfs/bbp.cscs.ch/ssd/apps/bb5/systemtools:/opt/c3/bin:/opt/ibutils/bin:/opt/ddn/ime/bin:/sbin:/bin:/gpfs/bbp.cscs.ch/home/mandge/.local/bin:/gpfs/bbp.cscs.ch/home/mandge/bin)
/gpfs/bbp.cscs.ch/home/mandge/lfpy/multimodalfitting/ecode
Mod files: "mechanisms/CaDynamics_E2.mod" "mechanisms/Ca_HVA.mod" "mechanisms/Ca_LVAst.mod" "mechanisms/epsp.mod" 

In [3]:
probe = None #model.define_electrode(probe_type="linear")

In [4]:
cell = model.create(morph_modifier="hillock", release=True)

param_names = [param.name for param in cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(
    feature_set=None, 
    feature_file=None, 
    electrode=probe,
    protocols_with_lfp=None
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [5]:
evaluator.get_protocol_definitions()
# # print(eva)
# !pwd

{'bAP': {'stimuli': [{'delay': 95,
    'amp': 1.9,
    'duration': 5,
    'totduration': 400}],
  'extra_recordings': [{'var': 'v',
    'somadistance': 400,
    'type': 'somadistance',
    'name': 'dend1',
    'seclist_name': 'apical'},
   {'var': 'v',
    'somadistance': 620,
    'type': 'somadistance',
    'name': 'dend2',
    'seclist_name': 'apical'}]},
 'BAC': {'stimuli': [{'delay': 95,
    'amp': 1.9,
    'duration': 5,
    'totduration': 400}],
  'extra_recordings': [{'var': 'v',
    'somadistance': 400,
    'type': 'somadistance',
    'name': 'dend1',
    'seclist_name': 'apical'},
   {'var': 'v',
    'somadistance': 620,
    'type': 'somadistance',
    'name': 'dend2',
    'seclist_name': 'apical'}]},
 'EPSP': {'stimuli': [{'delay': 95,
    'amp': 1.9,
    'duration': 5,
    'totduration': 400}],
  'extra_recordings': [{'var': 'v',
    'somadistance': 400,
    'type': 'somadistance',
    'name': 'dend1',
    'seclist_name': 'apical'},
   {'var': 'v',
    'somadistance': 620,
 

In [6]:
# evaluator.define_protocols()
allprots = evaluator.define_all_protocols()

In [ ]:
t_start = time.time()
# responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values={})
responses = eva.run_protocols(protocols=allprots.values(), param_values={}) 
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
responses

In [ ]:
fig, axes = plt.subplots(len(responses), figsize=(10, 20), squeeze=False)
for index, (resp_name, response) in enumerate(sorted(responses.items())):
    axes[index, 0].plot(response['time'], response['voltage'], label=resp_name)
    axes[index, 0].set_title(resp_name)
fig.tight_layout()
fig.show()

In [ ]:
fig.savefig("hay_optimization_validation_protocols.png")

In [ ]:
fig2, axes2 = plt.subplots(3, figsize=(10, 5), squeeze=False)

axes2[0, 0].plot(responses['sAHP.soma.v']['time'], responses['sAHP.soma.v']['voltage'], label='sAHP.soma.v')
axes2[0, 0].set_title('sAHP.soma.v')
axes2[1, 0].plot(responses['HyperDepol.soma.v']['time'], responses['HyperDepol.soma.v']['voltage'], label='HyperDepol.soma.v')
axes2[1, 0].set_title('HyperDepol.soma.v')
axes2[2, 0].plot(responses['PosCheops.soma.v']['time'], responses['PosCheops.soma.v']['voltage'], label='PosCheops.soma.v')
axes2[2, 0].set_title('PosCheops.soma.v')
# axes2[3, 0].plot(responses['NoiseOU3.soma.v']['time'], responses['NoiseOU3.soma.v']['voltage'], label='NoiseOU3.soma.v') # Noise protocol if added to protocols
# axes2[3, 0].set_title('NoiseOU3.soma.v')

fig2.tight_layout()
fig2.show()